In this Jupyter notebook, we generate a model that's able to successfully predict whether a molecule will bind to a specific target. For the first iteration of this notebook, we'll focus on predicting whether or not molecules will bind to the 5-hydroxytryptamine receptors (5-HTR). We draw inspiration from the following paper: https://pdfs.semanticscholar.org/c108/970bcb96967af5f5ba2783738bd39e751725.pdf

Let's start by importing the necessary dependencies.

In [1]:
# Base packages
import os
import settings
import string
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import interp
import matplotlib.pyplot as plt
from google.cloud import bigquery

# Scikit-learn packages
from sklearn.base import clone
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_curve, auc
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# RDkit, a cheminformatics library
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import PandasTools
from rdkit.Chem.Draw import IPythonConsole

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

Our analysis dataset will be generated by querying the ChEMBL database. This database is publicly available through the Google Cloud Platform (GCP). To access this platform through Python, we need to configure our Python environment and define the location of the GCP credentials.

In [2]:
# Define relative paths
NOTEBOOKS = os.getcwd()
WKDIR = NOTEBOOKS.replace('/Notebooks', '')
DATA = WKDIR + 'Data'

# Define location of credentials and initialize client
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = WKDIR + "/" + settings.GOOGLE_CLOUD_CREDENTIALS
EBI_CHEMBL = "patents-public-data.ebi_chembl"
client = bigquery.Client()

Let's first define the specific targets whose binding profile we want to characterize with our model.

In [3]:
target_query = f"""
SELECT DISTINCT 
    pref_name
FROM 
    `{EBI_CHEMBL}.target_dictionary_24`
WHERE 
    pref_name like "%5-HT%"   AND
    organism = "Homo sapiens" AND
    target_type = "SINGLE PROTEIN"
"""
target_list = client.query(target_query).to_dataframe()
target_list

,pref_name
0,Serotonin 1a (5-HT1a) receptor
1,Serotonin 1d (5-HT1d) receptor
2,Serotonin 1b (5-HT1b) receptor
3,Serotonin 2a (5-HT2a) receptor
4,Serotonin 2c (5-HT2c) receptor
5,Serotonin 3a (5-HT3a) receptor
6,Serotonin 4 (5-HT4) receptor
7,Serotonin 2b (5-HT2b) receptor
8,Serotonin 1f (5-HT1f) receptor
9,Serotonin 7 (5-HT7) receptor


In order to measure the biological activity of a given molecule, researchers perform assays, or experimental procedures that attempt to characterize a particular behavior or quantify a specific property associated with the entity of interest. For the purposes of our model, we’re interested in assays that measure drug affinity, or the concentration of the drug needed to induce some sort of observable effect. Let’s first identify the most common measurements associated with binding assays.

In [4]:
ACTIVITIES = f"{EBI_CHEMBL}.activities_24"
ACT_LOOKUP = f"{EBI_CHEMBL}.activity_stds_lookup_24"
ASSAYS = f"{EBI_CHEMBL}.assays_24"
CMPD_STR = f"{EBI_CHEMBL}.compound_structures_24"
MOL_DICT = f"{EBI_CHEMBL}.molecule_dictionary_24"
TGT_DICT = f"{EBI_CHEMBL}.target_dictionary_24"

binding_act_qtext = f"""
SELECT
    act.standard_type        AS standard_type,
    COUNT(act.standard_type) AS count,
    act_std.definition       AS definition
FROM `{ACTIVITIES}` act
    INNER JOIN `{ASSAYS}`     assays  on assays.assay_id = act.assay_id
    INNER JOIN `{ACT_LOOKUP}` act_std on act_std.standard_type = act.standard_type
WHERE 
    assays.assay_type = "B" AND 
    act_std.standard_units = "nM"
GROUP BY 
    standard_type,
    definition
ORDER BY count DESC
"""
binding_act = client.query(binding_act_qtext).to_dataframe()
binding_act

,standard_type,count,definition
0,IC50,1151059,Concentration required for 50% inhibition
1,Ki,572328,Inhibition constant
2,Potency,245136,Concentration or dose required to elicit a spe...
3,Kd,76349,Dissociation constant
4,EC50,72629,Effective concentration for 50% activity
5,AC50,10584,Concentration required for 50% activity
6,Km,6573,Michaelis constant
7,Kb,2493,Equilibrium binding constant
8,MIC,1596,Minimum inhibitory concentration
9,GI50,583,Concentration required for 50% growth inhibition


These results seem to make sense. For example, EC50 (the fifth most common measurement in the table above) represents the concentration of a molecule needed to induce half the maximal response. A low EC50 implies that a small concentration of this molecule induces half the maximal observable response, which provides strong evidence that the molecule actually binds to the target of interest. Conversely, a high EC50 indicates that the molecule doesn’t display high affinity for the target.

While we could simply define our query to only return activity measurements produced by binding assays, let’s first confirm that other, non-binding assays are not associated with these kinds of measurements.

In [5]:
# Convert the measurements above into a list, and remove solubility
measurements = binding_act['standard_type'].tolist()
measurements = measurements[:-1]

# Convert the measurements into a string for the SQL query
measurements_str = ', '.join(f"'{m}'" for m in measurements)

# Define and execute query
ASSAY_TYPE = f"{EBI_CHEMBL}.assay_type_24"
assay_measures_qtext = f"""
SELECT 
    assays.assay_type        AS assay_type,
    COUNT(assays.assay_type) AS count,
    type.assay_desc          AS assay_desc
FROM `{ASSAYS}` assays
    INNER JOIN `{ACTIVITIES}` act on act.assay_id = assays.assay_id
    INNER JOIN `{ASSAY_TYPE}` type on type.assay_type = assays.assay_type
WHERE 
    act.standard_type in ({measurements_str})
GROUP BY
    assay_type,
    assay_desc
ORDER BY count DESC
"""
assay_measures = client.query(assay_measures_qtext).to_dataframe()
assay_measures

,assay_type,count,assay_desc
0,F,8804059,Functional
1,B,2140145,Binding
2,A,142813,ADME
3,T,14363,Toxicity
4,P,2529,Physicochemical
5,U,674,Unassigned


Interesting. It seems that functional assays also produce activity measurements that could be relevant for our model. Let’s take a look at some of the documents (i.e., journal articles) associated with these non-binding assays to confirm that they actually reference molecular binding or molecular potency.

In [6]:
DOCUMENTS = f"{EBI_CHEMBL}.docs_24"
docs_nbind_qtext = f"""
SELECT DISTINCT
    assays.assay_type AS assay_type,
    act.standard_type AS standard_type,
    docs.title        AS title,
    docs.abstract     AS abstract
FROM `{DOCUMENTS}` docs
    INNER JOIN `{ASSAYS}` assays  on assays.doc_id = docs.doc_id
    INNER JOIN `{ACTIVITIES}` act on act.assay_id = assays.assay_id
WHERE
    assays.assay_type != "B"                  AND
    act.standard_type in ({measurements_str}) AND
    docs.title IS NOT NULL                    AND
    docs.abstract IS NOT NULL
ORDER BY
    assays.assay_type,
    act.standard_type
"""
docs_nbind = client.query(docs_nbind_qtext).to_dataframe()
docs_nbind

,assay_type,standard_type,title,abstract
0,A,AC50,Identification of biphenyl-based hybrid molecu...,With the aim of enhancing the structural compl...
1,A,AC50,"Antitumor agents from bohemic acid complex, II...","Six anthracycline antitumor agents, marcellomy..."
2,A,AC50,Neuroprotective activity and evaluation of Hsp...,Alzheimer's disease (AD) neuropathology is cha...
3,A,AC50,Didemnins and tunichlorin: novel natural produ...,"Didemnins A, B, and C, cyclic depsipeptides pr..."
4,A,AC50,Exploration of the correlation between the str...,The hemolytic activity of a collection of 63 s...
...,...,...,...,...
41357,U,IC50,Cytotoxic and apoptosis-inducing activities of...,"Six lanostane-type triterpene acids (1a-6a), i..."
41358,U,IC50,Bioactive constituents of Cedrelopsis microfol...,The stem bark of Cedrelopsis microfoliata has ...
41359,U,Ki,Vitamin B6s inhibit oxidative stress caused by...,Cu(II) complexes of Alzheimer's disease-relate...
41360,U,Km,Selenoureido-iminosugars: A new family of mult...,Herein we report the synthesis of N-alkylated ...


After exploring a document associated with each of the different assay types (linked below), it's clear that non-binding assays can measure binding activity.

- ADME assay: https://www.ncbi.nlm.nih.gov/pubmed/19072652
- Functional assay: https://www.ncbi.nlm.nih.gov/pubmed/26555041
- Physicochemical assay: https://www.ncbi.nlm.nih.gov/pubmed/25752525
- Toxicity assay: https://www.ncbi.nlm.nih.gov/pubmed/27639365
- Unassigned assay: https://www.ncbi.nlm.nih.gov/pubmed/19699641

Having determined the measurements of interest, let's start by characterizing the binding profile of the serotonin 1a receptor.

In [7]:
ACTIVITIES = f"{EBI_CHEMBL}.activities_24"
ASSAYS = f"{EBI_CHEMBL}.assays_24"
CMPD_STR = f"{EBI_CHEMBL}.compound_structures_24"
MOL_DICT = f"{EBI_CHEMBL}.molecule_dictionary_24"
TGT_DICT = f"{EBI_CHEMBL}.target_dictionary_24"

analysis_qtext = f"""
SELECT DISTINCT
    mol_dict.chembl_id        AS mol_chembl_id,
    act.molregno              AS molregno,
    act.standard_type         AS standard_type,
    act.standard_relation     AS standard_relation,
    act.standard_value        AS standard_value,
    act.standard_units        AS standard_units,
    cmpd_str.canonical_smiles AS canonical_smiles,
    tgt_dict.chembl_id        AS tgt_chembl_id,
    tgt_dict.pref_name        AS tgt_name,
    assays.chembl_id          AS assay_chembl_id,
    assays.description        AS assay_desc
FROM `{ACTIVITIES}` act
    INNER JOIN `{MOL_DICT}` mol_dict ON mol_dict.molregno = act.molregno
    INNER JOIN `{ASSAYS}`   assays   ON assays.assay_id = act.assay_id
    INNER JOIN `{TGT_DICT}` tgt_dict ON tgt_dict.tid = assays.tid
    INNER JOIN `{CMPD_STR}` cmpd_str ON cmpd_str.molregno = act.molregno
WHERE
    act.standard_units = "nM"               AND
    act.standard_relation in ("<", "=")     AND
    act.potential_duplicate = '0'           AND
    assays.confidence_score >= '8'          AND
    tgt_dict.organism = "Homo sapiens"      AND
    tgt_dict.target_type = "SINGLE PROTEIN" AND
    tgt_dict.pref_name = "Serotonin 1a (5-HT1a) receptor" AND
    act.standard_type IN ({measurements_str})
"""
data = client.query(analysis_qtext).to_dataframe()

# Convert the standard value to a numeric variable
data['standard_value'] = pd.to_numeric(data['standard_value'])

# Save out a CSV
data.to_csv(DATA + '/ChEMBL Activity data (5-HT1a).csv', index=False)
data.head()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/trivedim/Desktop/ML_projects/Binding_clfData/ChEMBL Activity data (5-HT1a).csv'

There are 814 duplicate molecules, and there are 12 duplicate molecule-activity values. In order to handle the latter, we will simply drop the duplicates. In order to handle the former, we will keep the molecule associated with the lower activity value.

In [ ]:
print("No. of duplicate molecules: " + str(data.duplicated(subset=['mol_chembl_id']).sum()))
print("No. of duplicate molecule-activity values: " + str(data.duplicated(subset=['mol_chembl_id', 'standard_value']).sum()))

# Handle duplicate molecule-activity values
data_nodup = data.drop_duplicates(subset=['mol_chembl_id', 'standard_value'], inplace=False)

# Handle duplicate molecules
assert data_nodup.duplicated(subset=['mol_chembl_id', 'standard_value']).sum()==0
data_nodup.sort_values(by=['standard_value', 'mol_chembl_id'], ascending=True, inplace=True)
data_nodup.drop_duplicates(subset=['mol_chembl_id'], keep='first', inplace=True)
assert data_nodup.duplicated(subset=['mol_chembl_id']).sum()==0

# Save out a dataset without duplicates
data_nodup.to_csv(DATA + '/ChEMBL Activity Data (5-HT1a, no dup).csv', index=False)
print("Data dimensions after removing duplicates: " + str(data_nodup.shape))

We assume that a molecule binds to the target if its activity value (i.e., the concentration needed to induce a response) is less than or equal to 100nM. This threshold is based on the activity threshold for GPCRs here: https://druggablegenome.net/ProteinFam

In [ ]:
def set_active(row):
    active = 0
    if row['standard_value'] <= 100:
        active = 1
    return active

# Create the 'active' variable in both the duplicated and non-duplicated datasets
data['active'] = data.apply(lambda x: set_active(x), axis=1)
data_nodup['active'] = data_nodup.apply(lambda x: set_active(x), axis=1)

# View data
data_nodup.head()

The probability that a given molecule binds to a target is dependent on, among other factors, the structure of the molecule itself. We can encode the structures of the molecules in our datasets using circular fingerprints. Circular fingerprints flatten the 2-D molecule structure into a 1-D vector by exhaustively enumerating and hashing all circular fragments grown radially from each heavy atom of the molecule up to a given radius. Figure 1 below illustrates this algorithm:

**Figure 1: Circular fingerprints**

<img src="CircularEnumeration.png">

Source: https://docs.eyesopen.com/toolkits/python/graphsimtk/fingerprint.html

In [ ]:
def generate_fingerprint_matrix(compounds, radius, bit_length):
    """Create an {M x N} matrix encoding the Morgan fingerprints for the inputted list of compounds.
    
    Keyword arguments:
    compounds  -- a list of the SMILES associated with M compounds for which to generate Morgan fingerprints 
    radius     -- the radius used to generate the fingerprint
    bit_length -- the number of bits N used to encode each fingerprint
    """
    
    # Define the dimensions of the Morgan matrix
    m = len(compounds)
    n = bit_length
    
    # Initialize a {1 x N} row
    fp_matrix = np.zeros((1, bit_length))

    # Iterate through the compounds and generate the fingerprint for each
    for c in range(0, m):
        cmpd_smile = compounds[c]
        try:
            mol = Chem.MolFromSmiles(cmpd_smile)
            fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=bit_length)
            fp_vector = np.array([int(x) for x in list(fp.ToBitString())])
            fp_matrix = np.row_stack((fp_matrix, fp_vector))
            
            # For tracking progress
            if c%500==0:
                print(f"Progress percentage: {np.round(100*(c/m),2)}%")

        except:
            print(f"Unable to generate fingerprint for molecule {c}")
        
    # Delete the first row of zeros
    fp_matrix = np.delete(fp_matrix, 0, axis=0)
    
    # Return the matrix
    print(f"Dimensions of matrix: {fp_matrix.shape}")
    return fp_matrix 

In [ ]:
smiles = data_nodup['canonical_smiles'].tolist()
fp_matrix = generate_fingerprint_matrix(smiles, 2, 1024)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(fp_matrix, data_nodup['active'], test_size=0.33, random_state=42)
print(f"Dimensions of X_train: {X_train.shape}")
print(f"Dimensions of X_test: {X_test.shape}")
print(f"Dimensions of y_train: {y_train.shape}")
print(f"Dimensions of y_test: {y_test.shape}")